In [2]:
from langchain.document_loaders import UnstructuredHTMLLoader

In [3]:
loaders = [
    UnstructuredHTMLLoader("example2.html"),
    UnstructuredHTMLLoader("example3.html"),
    UnstructuredHTMLLoader("example4.html"),
    UnstructuredHTMLLoader("example5.html"),
    UnstructuredHTMLLoader("example6.html"),
    UnstructuredHTMLLoader("example7.html"),
    UnstructuredHTMLLoader("example8.html"),
    UnstructuredHTMLLoader("example9.html"),
    UnstructuredHTMLLoader("example10.html"),
    UnstructuredHTMLLoader("example11.html"),
    UnstructuredHTMLLoader("example12.html"),
    UnstructuredHTMLLoader("example13.html")
]
docs = []
for l in loaders:
    docs.extend(l.load())

In [4]:
import os
#from openai.api_resources import engine
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "..."
os.environ["OPENAI_API_BASE"] = "..."

In [5]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
#raw_documents = TextLoader('../../../state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings()
#embeddings = HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl")
db = FAISS.from_documents(documents, embeddings)

Created a chunk of size 2459, which is longer than the specified 1000


In [6]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [7]:
retriever = db.as_retriever()

docs = retriever.get_relevant_documents("What are positive, negative and neutral news about Microsoft?")
pretty_print_docs(docs)

Document 1:

Microsoft was rich, powerful and ambitious. It had started to move beyond software. In 1996, it entered the media business in a partnership with NBC, setting up the cable channel, MSNBC, and a website, msnbc.com (Microsoft shed its stakes in both, years later).

Executives in industry after industry worried about what Microsoft might do next, sharing a sentiment expressed by Rupert Murdoch, chairman of News Corp: “Everybody in the communications business is paranoid of Microsoft, including me.”

Today, Google does not loom as large as Microsoft once did. It is one member of the Big Tech club. It is an undisputed giant in search and online advertising, and its software and artificial intelligence prowess can extend to other industries. But its Big Tech peers — Amazon, Apple, Meta (Facebook) and even Microsoft — are all under scrutiny in the United States and abroad.
----------------------------------------------------------------------------------------------------
Document

In [8]:
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import AzureChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

llm = AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0.7)
#compressor = LLMChainExtractor.from_llm(llm)
#compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat

qa = RetrievalQA.from_chain_type(llm, chain_type='stuff', retriever=retriever)

#compressed_docs = compression_retriever.get_relevant_documents("What are positive, negative and neutral news about Microsoft?")
query = "What are positive, negative and neutral news about Microsoft? Give two examples of each"
qa.run(query)

"Positive News about Microsoft:\n1. Microsoft's revenue last quarter rose 10% year over year, despite a decline in demand for PCs and consumer tech devices. This growth was primarily driven by the success of its cloud and enterprise services, particularly the Azure platform.\n2. Microsoft consistently generates a high operating profit margin, with over 40% of its sales turning into operating profit. In the most recent quarter, this key metric jumped an impressive 21%, showcasing the company's financial strength.\n\nNegative News about Microsoft:\n1. Microsoft was found to have repeatedly violated the nation's antitrust laws in the late 1990s. While an appeals court upheld most of the findings, the government's preferred remedy of breaking up the company was not implemented.\n2. The media industry expressed concerns and paranoia about Microsoft's ambitions and potential dominance in various sectors, with industry executives worrying about what Microsoft might do next.\n\nNeutral News ab